In [1]:
#Download and unzip the dataset

# download dataset from the UCI website
!curl -o uci-labelled-sentences.zip https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84188    0 84188    0     0   293k      0 --:--:-- --:--:-- --:--:--  293k


In [2]:

# unzip dataset in Colab
!unzip uci-labelled-sentences.zip

Archive:  uci-labelled-sentences.zip
   creating: sentiment labelled sentences/
  inflating: sentiment labelled sentences/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/sentiment labelled sentences/
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


In [3]:

import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
df_list = []

# Yelp
df_yelp = pd.read_csv('sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df_yelp['source'] = 'yelp'
df_list.append(df_yelp)

# Amazon
df_amazon = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', names=['sentence', 'label'], sep='\t')
df_amazon['source'] = 'amazon'
df_list.append(df_amazon)

# IMDB
df_imdb = pd.read_csv('sentiment labelled sentences/imdb_labelled.txt', names=['sentence', 'label'], sep='\t')
df_imdb['source'] = 'imdb'
df_list.append(df_imdb)

# Concatenate the dataframes
df = pd.concat(df_list)
df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [5]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['sentence'].values)
X = tokenizer.texts_to_sequences(df['sentence'].values)
X = pad_sequences(X)
y = df['label'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12)

In [7]:
def create_model(embed_dim = 64):
  model = Sequential()
  model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
  model.add(LSTM(16))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = create_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test),callbacks = [EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)])


Epoch 1/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 85s 540ms/step - accuracy: 0.5635 - loss: 0.6796 - val_accuracy: 0.7515 - val_loss: 0.5356
Epoch 2/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 142s 543ms/step - accuracy: 0.8507 - loss: 0.3900 - val_accuracy: 0.7818 - val_loss: 0.4731
Epoch 3/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 143s 549ms/step - accuracy: 0.9307 - loss: 0.1998 - val_accuracy: 0.7909 - val_loss: 0.4788
Epoch 4/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 145s 573ms/step - accuracy: 0.9617 - loss: 0.1365 - val_accuracy: 0.7970 - val_loss: 0.5699
Epoch 5/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 139s 552ms/step - accuracy: 0.9793 - loss: 0.0805 - val_accuracy: 0.7939 - val_loss: 0.6464
Epoch 6/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 85s 561ms/step - accuracy: 0.9834 - loss: 0.0607 - val_accuracy: 0.8030 - val_loss: 0.7051


In [9]:
model.save("uci_sentimentanalysis.h5")

In [10]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.DEFAULT_PROTOCOL)

# New section

In [11]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence

In [12]:
model = load_model('uci_sentimentanalysis.h5')